In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
import face_recognition
import cv2
import time

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

In [4]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [5]:
from tensorflow.keras.models import load_model

# Load the pre-trained CNN model
model.load_weights('vgg_face_weights.h5')
vgg = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [6]:
def make_vgg(img1):
    img1 = cv2.resize(img1, (224, 224))
    img1 = np.array(img1).reshape(-1, 224, 224, 3)
    img1 = img1 / 255
    img1_representation = vgg.predict(img1, verbose=0)
    return img1_representation

In [7]:
def compare(img1, img2):
    euclidean_distance = np.linalg.norm(img1[0] - img2[0])
    if euclidean_distance < 0.45:
        return True
    else:
        return False

In [8]:
def compare2(img1, img2):
    euclidean_distance = np.linalg.norm(img1[0] - img2[0])
    return euclidean_distance

In [9]:
faces_dict = {}
cap = cv2.VideoCapture(0)

start_time = time.time()
count = 0
while True:
    _ , img = cap.read()
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_rect = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=10)
    
    elapsed_time = int(time.time() - start_time)
    cv2.putText(img, "{} seconds".format(elapsed_time), (10, 100), 0, 1, (255,255,255), 2)
    
    
    for (x,y,w,h) in face_rect:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), thickness = 2)
        face_gray = gray_img[y:y + h, x:x + w]
        face_color = img[y:y + h, x:x + w]
        smile_rect = smile_cascade.detectMultiScale(face_gray, scaleFactor=1.9, minNeighbors=20)
        
        for (sx, sy, sw, sh) in smile_rect:
            x = int(x+w/7)
            y = int(y-h/10)
#             cv2.rectangle(smile_color, (sx,sy), (sx+sw, sy+sh), (0,0,255), thickness=1)
            cv2.putText(img, "Thank You", (x, y), 0, 1, (255,255,255), 2)
#             cv2.imwrite("frame%d.jpg" % smile, smile_color) 
            

            if faces_dict == {}:
                faces_dict[count] = {}
                faces_dict[count]['face'] = img[y:y+h, x:x+w]
                faces_dict[count]['vgg'] = make_vgg(img[y:y+h, x:x+w])
                faces_dict[count]['time'] = 0
                faces_dict[count]['start_time'] = time.time()
                
                count+=1
                cv2.imwrite("frame%d.jpg" % count, face_color) 
            
            else:
                unique = True
                vgg_frame = make_vgg(img[y:y+h, x:x+w])
                for i in faces_dict:
                    if(compare(faces_dict[i]['vgg'], vgg_frame)):
                        unique = False
#                         print(unique)
                        faces_dict[i]['time'] = time.time() - faces_dict[i]['start_time']
                        break
                        
                if unique:
#                     print(unique)
                    faces_dict[count] = {}
                    faces_dict[count]['face'] = img[y:y+h, x:x+w]
                    faces_dict[count]['vgg'] = vgg_frame
                    faces_dict[count]['time'] = 0
                    faces_dict[count]['start_time'] = time.time()
                    count+=1
                    cv2.imwrite("frame%d.jpg" % count, face_color) 
                print(compare2(faces_dict[0]['vgg'], vgg_frame))
                    
    cv2.putText(img, str(count), (10,50), 0, 1, (255,255,255), 2)
                        
                
    cv2.imshow("frame", img)
    key = cv2.waitKey(10)

    if key == 27:
        break
    
    stop_time = time.time()
        
cap.release()
cv2.destroyAllWindows()

0.33079013
0.3533956
0.34460253
0.33713153
0.36013988
0.3203313
0.31812403
0.3306101
0.34842327
0.3255381
0.3218704
0.3348588
0.33859247
0.33387712
0.33330998
0.37003917
0.35204855
0.33876994
0.3026726
0.29974863
0.3142107
0.295631
0.27456346
0.29500094
0.29527888
0.305691
0.30186293
0.29565477
0.30441362
0.30612493
0.3024717
0.2895486
0.29191798
0.3092133
0.30797073
0.48301452
0.47123846
0.46005756
0.47861242
0.47422913
0.48917523
0.4808754
0.4692433
0.4716386
0.47234437
0.4719
0.4810549
0.48088557
0.4847998
0.48648778
0.48964012
0.48133025
0.4701869
0.48134056


In [10]:
def format_duration(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60

    duration = "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)
    return duration

In [11]:
duration = int(stop_time-start_time)
duration = format_duration(duration)
start_time = time.ctime(start_time)
stop_time = time.ctime(stop_time)

print(duration, start_time, stop_time)

00:00:38 Mon Jun 12 16:46:49 2023 Mon Jun 12 16:47:28 2023


In [12]:
import pymongo

In [13]:
if __name__ == "__main__":
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client['Smile_Count']
    collection1 = db['Details']
    collection2 = db['Face_Details']
    details = {'start_time': start_time, 'stop_time': stop_time, 'duration': duration, 'Faces count': count}
    collection1.insert_one(details)
    
    toAdd = []
    
    for i in faces_dict:
        start = faces_dict[i]['start_time']
        start = time.ctime(start)
        time_duration = int(faces_dict[i]['time'])
        time_duration = format_duration(time_duration)
        toAdd.append({'start_time': start, 'duration': time_duration})
        
    collection2.insert_many(toAdd)

In [14]:
faces_dict

{0: {'face': array([[[158, 164, 163],
          [157, 164, 163],
          [158, 166, 165],
          ...,
          [141, 153, 155],
          [144, 155, 157],
          [145, 154, 158]],
  
         [[156, 164, 163],
          [156, 164, 163],
          [157, 165, 164],
          ...,
          [142, 153, 157],
          [145, 155, 159],
          [146, 155, 159]],
  
         [[156, 164, 164],
          [156, 164, 165],
          [157, 165, 166],
          ...,
          [143, 154, 158],
          [144, 154, 158],
          [147, 156, 160]],
  
         ...,
  
         [[112, 109, 118],
          [109, 105, 114],
          [104, 101, 110],
          ...,
          [ 65,  62,  73],
          [ 63,  63,  73],
          [ 61,  63,  73]],
  
         [[117, 117, 127],
          [112, 113, 123],
          [100, 101, 111],
          ...,
          [ 63,  63,  73],
          [ 62,  63,  73],
          [ 60,  62,  72]],
  
         [[127, 127, 137],
          [124, 125, 135],
          [10